## Tsetlin Machine trains on IMDB
This notebook shows how the green-tsetlin Tsetlin Machine trains on the IMDB sentiment dataset. With this tutorial, huggingface datasets library is used. The library holds the dataset that is on front preprocessed, removing symbols and white spaces.  

#### First we install the green-tsetlin using pip
The green-tsetlin library offers cpu heavy and less cpu heavy implemenation of the library, offering systems with older cpus a plug-and-play version of the library

In [39]:
#pip install green-tsetlin
#pip install green-tsetlin[cpu]

### Libraries

As the Tsetlin Machine trains on binary data, all input must be on binary form. Given a feature vector v : $[f_0, f_1, f_2]$ , this

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import green_tsetlin as gt
import datasets
import numpy as np
import os
import optuna

/home/kolla/anaconda3/envs/gt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
seed = 42
rng = np.random.default_rng(seed)

In [3]:
imdb = datasets.load_dataset('imdb')
x, y = imdb['train']['text'], imdb['train']['label']

In [37]:
print(x[0])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [4]:
vectorizer = CountVectorizer(ngram_range=(1, 1), binary=True, lowercase=True, max_features=5000)
vectorizer.fit(x)

x_bin = vectorizer.transform(x).toarray().astype(np.uint8)
y = np.array(y).astype(np.uint32)

In [6]:
shuffle_index = [i for i in range(len(x))]
rng.shuffle(shuffle_index)

x_bin = x_bin[shuffle_index]
y = y[shuffle_index]

train_x_bin, val_x_bin, train_y, val_y = train_test_split(x_bin, y, test_size=0.2, random_state=seed, shuffle=True)

train_x_bin = train_x_bin[:1000]
train_y = train_y[:1000]
val_x_bin = val_x_bin[:100]
val_y = val_y[:100]

In [7]:
n_clauses = 100
s = 2.0
threshold = 20
literal_budget = 5

n_epochs = 20


tm = gt.TsetlinMachine(n_literals=train_x_bin.shape[1], 
                       n_clauses=n_clauses, 
                       n_classes=len(np.unique(train_y)),
                       s=s,
                       threshold=threshold,
                       literal_budget=literal_budget)

trainer = gt.Trainer(tm=tm, 
                     n_jobs=int(os.cpu_count()-2),
                     n_epochs=n_epochs,
                     seed=seed,
                     progress_bar=True)

trainer.set_train_data(train_x_bin, train_y)
trainer.set_test_data(val_x_bin, val_y)

trainer.train()

Processing epoch 1 of 20, train acc: NA, best test score: NA:   0%|          | 0/20 [00:00<?, ?it/s]

Processing epoch 20 of 20, train acc: 0.514, best test score: 0.510 (epoch: 5): 100%|██████████| 20/20 [00:06<00:00,  3.26it/s]


{'train_time_of_epochs': [0.27510358399740653,
  0.2807319879975694,
  0.2641137769969646,
  0.3021336029996746,
  0.2633360769978026,
  0.2642809770004533,
  0.25833277399942745,
  0.2626812769995013,
  0.2592988750002405,
  0.2635421770028188,
  0.2592279739983496,
  0.2746862849999161,
  0.262940377000632,
  0.26281177700002445,
  0.3115096090004954,
  0.3865282590013521,
  0.4109816799973487,
  0.4026439779991051,
  0.3877953669980343,
  0.3716297559985833],
 'best_test_score': 0.51,
 'best_test_epoch': 5,
 'n_epochs': 20,
 'train_log': [0.488,
  0.514,
  0.514,
  0.486,
  0.481,
  0.49,
  0.502,
  0.468,
  0.5,
  0.527,
  0.481,
  0.501,
  0.492,
  0.493,
  0.499,
  0.506,
  0.501,
  0.508,
  0.5,
  0.514],
 'test_log': [0.47,
  0.42,
  0.47,
  0.48,
  0.45,
  0.51,
  0.45,
  0.44,
  0.4,
  0.45,
  0.42,
  0.47,
  0.43,
  0.47,
  0.44,
  0.44,
  0.48,
  0.42,
  0.44,
  0.43],
 'did_early_exit': False}

In [9]:
trainer.results.keys()

dict_keys(['train_time_of_epochs', 'best_test_score', 'best_test_epoch', 'n_epochs', 'train_log', 'test_log', 'did_early_exit'])

In [32]:
test_x, test_y = imdb['test']['text'], imdb['test']['label']

test_x_bin = vectorizer.transform(test_x).toarray().astype(np.uint8)
test_y = np.array(test_y).astype(np.uint32)

shuffle_index = [i for i in range(len(test_x))]
rng.shuffle(shuffle_index)

test_x_bin = test_x_bin[shuffle_index]
test_y = test_y[shuffle_index]

test_x_bin = test_x_bin[:100]
test_y = test_y[:100]

In [36]:
features = vectorizer.get_feature_names_out()
predictor = tm.get_predictor(explanation="none")
predictor.set_explanation_names(features)

for i in range(len(test_x_bin)):
    
    pred = predictor.explain(test_x_bin[i])
    print(pred)
    break

ValueError: Cannot request explanation on a predictor that has explanation set to 'none'.